## Анализ распространения Covid-19 и его воздействия на мировую экономику


Набор данных для анализа загружен с Kaggle и содержит следующие данные:
1. Код страны
2. Название страны
3. Дата записи
4. Индекс человеческого развития
5. Ежедневные случаи covid-19
6. Ежедневная смертность из-за covid-19
7. Индекс строгости
8. Население
9. ВВП на душу населения

Импортируем необходимые библиотеки Python и данные:

In [334]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

data2 = pd.read_csv("raw_data.csv")

## Подготовка данных

Набор данных содержит файл с необработанными данными о случаях заболевания Covid-19
и их влиянии на ВВП с 31 декабря 2019 года по 10 октября 2020 года

In [335]:
data2

,iso_code,location,date,total_cases,total_deaths,stringency_index,population,gdp_per_capita,human_development_index,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0.00,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0.00,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0.00,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0.00,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0.00,38928341,1803.987,0.498,#NUM!,#NUM!,#NUM!,17.477233,7.497754494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50413,ZWE,Zimbabwe,2020-10-15,8055.0,231.0,76.85,14862927,1899.775,0.535,8.994048296,5.442417711,4.34185547,16.514381,7.549490737
50414,ZWE,Zimbabwe,2020-10-16,8075.0,231.0,76.85,14862927,1899.775,0.535,8.996528148,5.442417711,4.34185547,16.514381,7.549490737
50415,ZWE,Zimbabwe,2020-10-17,8099.0,231.0,76.85,14862927,1899.775,0.535,8.999495876,5.442417711,4.34185547,16.514381,7.549490737
50416,ZWE,Zimbabwe,2020-10-18,8110.0,231.0,76.85,14862927,1899.775,0.535,9.000853147,5.442417711,4.34185547,16.514381,7.549490737


Создадим новый набор данных. Для этого посмотрим сколько образцов каждой страны присутствует в наборе данных:

In [336]:
data2["location"].value_counts()

Afghanistan        294
Indonesia          294
Macedonia          294
Luxembourg         294
Lithuania          294
                  ... 
Tajikistan         172
Comoros            171
Lesotho            158
Hong Kong           51
Solomon Islands      4
Name: location, Length: 210, dtype: int64

Получаем разное количество выборок каждой страны. Определим наиболее распространенное значение количества выборок:

In [337]:
data2["location"].value_counts().mode()

0    294
Name: location, dtype: int64

Т.о. 294 — наша отправная точка. Это значение будем использовать для деления суммы всех выборок, относящихся к индексу человеческого развития,
ВВП на душу населения и численности населения. Теперь создадим новый набор данных:

In [338]:
code = data2["iso_code"].unique().tolist()
country = data2["location"].unique().tolist()
hdi = []
tc = []
td = []
sti = []
population = data2["population"].unique().tolist()
gdp = []

for i in country:
    hdi.append((data2.loc[data2["location"] == i, "human_development_index"]).sum()/294)
    tc.append((data2.loc[data2["location"] == i, "total_cases"]).max())
    td.append((data2.loc[data2["location"] == i, "total_deaths"]).max())
    sti.append((data2.loc[data2["location"] == i, "stringency_index"]).sum()/294)
    gdp.append((data2.loc[data2["location"] == i, "gdp_per_capita"]).sum()/294)

aggregated_data = pd.DataFrame(list(zip(code, country, hdi, tc, td, sti, population, gdp)),
                               columns = ["Code", "Country", "HDI",
                                          "Total Cases", "Total Deaths",
                                          "Stringency Index", "Population", "GDP"])
aggregated_data.head()

,Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population,GDP
0,AFG,Afghanistan,0.498000,40200.0,1492.0,46.800034,38928341,1803.987000
1,ALB,Albania,0.600765,17055.0,451.0,48.625782,2877800,9033.238010
2,DZA,Algeria,0.754000,54402.0,1856.0,55.053231,43851043,13913.839000
3,AND,Andorra,0.659551,3377.0,59.0,31.290510,77265,0.000000
4,AGO,Angola,0.418952,7462.0,241.0,53.217925,32866268,4196.370544


Cтолбец ВВП на душу населения для наших целей малоинформативен, заменим его на 2 столбца данных МВФ ВВП(ППС,$) за 2019 и 2020г.
Чтобы не собирать вручную данные о ВВП всех стран, для изучения экономических последствий создадим подвыборку из
10 стран с наибольшим количеством случаев заболевания Covid-19

In [339]:
sample = aggregated_data.sort_values(by=["Total Cases"], ascending=False)
data = sample.head(10).drop(columns="GDP")
data

,Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population
200,USA,United States,0.924000,8154595.0,219674.0,51.031497,331002647
90,IND,India,0.640000,7550273.0,114610.0,61.329048,1380004385
27,BRA,Brazil,0.759000,5235344.0,153905.0,50.903469,212559409
157,RUS,Russia,0.816000,1399334.0,24187.0,50.636259,145934460
7,ARG,Argentina,0.707143,989667.0,26267.0,65.453333,45195777
42,COL,Colombia,0.581847,959572.0,28970.0,61.430884,50882884
178,ESP,Spain,0.887969,936560.0,33775.0,49.858163,46754783
68,FRA,France,0.901000,897034.0,33477.0,46.550884,65273512
150,PER,Peru,0.599490,868675.0,33759.0,65.919966,32971846
125,MEX,Mexico,0.774000,851227.0,86167.0,50.945136,128932753


Добавим в prepared_data два столбца - ВВП на душу населения до и во время Covid-19 (данные о ВВП собираются вручную):

In [340]:

data["GDP Before Covid"] = [65052, 6966, 15424,
                            28495, 22997, 15633,
                            42600, 49769, 13323, 20767]
data["GDP During Covid"] = [63078, 6532, 14890,
                            28171, 20764, 14421,
                            38244, 46213, 11879, 19117]
data = data
data

,Code,Country,HDI,Total Cases,Total Deaths,Stringency Index,Population,GDP Before Covid,GDP During Covid
200,USA,United States,0.924000,8154595.0,219674.0,51.031497,331002647,65052,63078
90,IND,India,0.640000,7550273.0,114610.0,61.329048,1380004385,6966,6532
27,BRA,Brazil,0.759000,5235344.0,153905.0,50.903469,212559409,15424,14890
157,RUS,Russia,0.816000,1399334.0,24187.0,50.636259,145934460,28495,28171
7,ARG,Argentina,0.707143,989667.0,26267.0,65.453333,45195777,22997,20764
42,COL,Colombia,0.581847,959572.0,28970.0,61.430884,50882884,15633,14421
178,ESP,Spain,0.887969,936560.0,33775.0,49.858163,46754783,42600,38244
68,FRA,France,0.901000,897034.0,33477.0,46.550884,65273512,49769,46213
150,PER,Peru,0.599490,868675.0,33759.0,65.919966,32971846,13323,11879
125,MEX,Mexico,0.774000,851227.0,86167.0,50.945136,128932753,20767,19117


## Анализ распространения Covid-19

In [341]:
figure = px.bar(data, y='Total Cases', x='Country', title="Countries with Highest Covid Cases")
figure.show()

In [342]:
figure = px.bar(data, y='Total Deaths', x='Country', title="Countries with Highest Deaths")
figure.show()

In [343]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["Total Cases"],
    name='Total Cases',
))
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["Total Deaths"],
    name='Total Deaths',
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [344]:
cases = data["Total Cases"].sum()
deceased = data["Total Deaths"].sum()

labels = ["Total Cases", "Total Deaths"]
values = [cases - deceased, deceased]

fig = px.pie(data, values=values, names=labels, title='Percentage of Total Cases and Deaths', hole=0.5)
fig.show()

**Уровень смертности от случаев Covid-19:**

In [345]:
death_rate = (data["Total Deaths"].sum() / data["Total Cases"].sum()) * 100
print("Уровень смертности = ", death_rate)

Уровень смертности =  2.7109524539314864


Важным столбцом в этом наборе данных является индекс строгости. Это составная мера индикаторов реагирования, включая закрытие школ,
закрытие рабочих мест, запреты на масссовые мероприятия, поездки. Он показывает, насколько строго страны соблюдают следующие меры
по борьбе с распространением covid-19:

In [346]:
fig = px.bar(data, x='Country', y='Total Cases',
             hover_data=['Population', 'Total Deaths'],
             color='Stringency Index', height=400,
             title= "Stringency Index during Covid-19")
fig.show()

## Анализ воздействия Covid-19 на экономику

ВВП на душу населения до вспышки covid-19 среди 10 стран с наибольшим количеством случаев covid-19:

In [347]:
fig = px.bar(data, x='Country', y='Total Cases',
             hover_data=['Population', 'Total Deaths'],
             color='GDP Before Covid', height=400,
             title="GDP Per Capita Before Covid-19")
fig.show()

Сравним ВВП на душу населения до Covid-19 и во время Covid-19, чтобы оценить влияние covid-19 на ВВП на душу населения:

In [348]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["GDP Before Covid"],
    name='GDP Per Capita Before Covid-19',
))
fig.add_trace(go.Bar(
    x=data["Country"],
    y=data["GDP During Covid"],
    name='GDP Per Capita During Covid-19',
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

Во всех странах произошло падение ВВП на душу населения.

Важным экономическим фактором так же является индекс человеческого развития. Это интегральный показатель для межстранового сравнения и
измерения уровня жизни, грамотности, образованности и долголетия как основных характеристик человеческого потенциала исследуемой территории.

In [349]:
fig = px.bar(data, x='Country', y='Total Cases',
             hover_data=['Population', 'Total Deaths'],
             color='HDI', height=400,
             title="Human Development Index during Covid-19")
fig.show()

Влияние Human Development Index как фактора на распространение covid-19, а также на изменение ВВП на душу населения, не выявлено.

## Заключение

Используя средства pandas visualisation, на основе набора данных o covid-19 Kaggle мы проанализировали распространение covid-19 среди стран
и его влияние на экономику. Мы определили, в каких странах вспышка covid-19 привела к наибольшему количеству случаев covid-19 и смертей,
и какое влияние оказал на это индекс строгости. Мы также проанализировали, как вспышка covid-19 повлияла на показатель ВВП по ППС на душу
населения в каждой стране.